In [ ]:
import requests
from bs4 import BeautifulSoup
# we can use
# https://www.moneycontrol.com/news/business/
#
url = 'https://economictimes.indiatimes.com/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Look for all divs with class "clearfix"
articles = soup.find_all('li', class_='clearfix')

for article in articles:
    headline_tag = article.find('h2')
    if headline_tag:
        title = headline_tag.get_text(strip=True)
        link_tag = headline_tag.find('a')
        link = link_tag['href'] if link_tag else None

        print(f"Title: {title}")
        print(f"Link: {link}")
        print("-" * 50)

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL of Economic Times
url = "https://economictimes.indiatimes.com/"

# Send HTTP request to the URL
response = requests.get(url)

# Parse the content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all the news headlines (you can change the class based on the website structure)
headlines = soup.find_all('h2', {'class': 'heading'})

# Extract and print the headlines
for idx, headline in enumerate(headlines, 1):
    print(f"{idx}. {headline.get_text(strip=True)}")


In [ ]:
!pip install selenium
!pip install webdriver-manager

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape
url = "https://economictimes.indiatimes.com/news/newsblogs"

# Send HTTP request to the website
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all news blog titles and links
news_items = soup.find_all('div', class_='eachStory')

# Lists to hold the extracted news data
headlines = []
links = []

# Loop through each news item to get the title and link
for item in news_items:
    title = item.find('a')
    if title:
        headlines.append(title.get_text(strip=True))
        links.append("https://economictimes.indiatimes.com" + title.get('href'))

# Create a DataFrame to store the data
news_data = pd.DataFrame({
    'Headline': headlines,
    'Link': links
})

# Display the news data
print(news_data)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

url = "https://economictimes.indiatimes.com/news/newsblogs"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

news_items = soup.find_all('div', class_='eachStory')

news_articles = []
for item in news_items:
    title = item.find('a')
    if title:
        news_articles.append(title.get_text(strip=True))

df = pd.read_csv('/content/company_description.csv')

model = SentenceTransformer('all-MiniLM-L6-v2')

descriptions = df['Description'].tolist()
description_embeddings = model.encode(descriptions, convert_to_tensor=True)

def compute_ticker_scores(news_articles):
    ticker_scores = {ticker: 0 for ticker in df['Ticker']}

    for news in news_articles:
        news_embedding = model.encode(news, convert_to_tensor=True)
        cos_scores = util.cos_sim(news_embedding, description_embeddings)[0]

        for i, score in enumerate(cos_scores):
            ticker_scores[df.iloc[i]['Ticker']] += score.item()

    max_score = max(ticker_scores.values())
    min_score = min(ticker_scores.values())

    for ticker in ticker_scores:
        if max_score != min_score:
            normalized_score = 2 * (ticker_scores[ticker] - min_score) / (max_score - min_score) - 1
        else:
            normalized_score = 0
        ticker_scores[ticker] = normalized_score

    return ticker_scores

ticker_scores = compute_ticker_scores(news_articles)
sorted_scores = sorted(ticker_scores.items(), key=lambda x: x[1], reverse=True)

print("Top 10 Highest Influence Scores:")
for ticker, score in sorted_scores[:10]:
    print(f"Ticker: {ticker}, Score: {score:.4f}")

print("\nTop 10 Lowest Influence Scores:")
for ticker, score in sorted_scores[-10:]:
    print(f"Ticker: {ticker}, Score: {score:.4f}")